# [Heritiana Daniel Andriasolofo](handriasolofo@aimsammi.org)

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

In [ ]:
!rm data.zip train* valid*
!wget -O data.zip https://github.com/heritiana-aimsammi-sn2022/NLP_week1/blob/main/Lab1/data/data.zip?raw=true
!unzip data.zip
!rm data.zip

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [3]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [4]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [5]:
def softmax(x):
    ### FILL CODE
    max_val = max(x)
    x -= max_val
    exp_x = np.exp(x)
    return exp_x/np.sum(exp_x)

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

$\ell = - \log (f_y(wx))$

$\frac{\partial \ell}{\partial w_i} = -(\delta_{i, y} - f_i(wx))x$

In [6]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    for iter in range(niter):
        ### FILL CODE
        idx = np.random.randint(len(data))
        y, x = data[idx]
        f = softmax(w @ x)
        stoc_grad = np.zeros_like(w)
        for i in range(nlabels):
            delta_iy = 1 if i == y else 0
            stoc_grad[i] = -(delta_iy - f[i])*x
        w -= stoc_grad
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [7]:
def predict(w, x):
    ## FILL CODE
    z = w @ x
    label_pred = np.argmax(z)
    return label_pred  

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [8]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    y_true = [label for label,_ in valid_data]
    y_pred = [predict(w, x) for _,x in valid_data]
    n = len(y_true)
    accuracy = 0.0
    for i in range(n):
        if y_true[i] == y_pred[i]:
            accuracy += 1

    return accuracy/n

In [9]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros((nlabels, dim))
w = sgd(w, train_data, 5000)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 0.907

